In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.data_subject import DataSubject
from syft.core.tensor.smpc.mpc_tensor import MPCTensor
from syft.core.smpc.protocol.aby3 import ABY3
from syft.core.smpc.approximations import exp
import time
import operator
import uuid
sy.logger.remove()

/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DOMAIN1_PORT = 8081
DOMAIN2_PORT = 8082
p1 =  sy.login(email="info@openmined.org",password="changethis",port=DOMAIN1_PORT)
p2 = sy.login(email="info@openmined.org",password="changethis",port=DOMAIN2_PORT)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into italy... done!


In [3]:
ROOT_DIR = "/home/azureuser/PySyft"
def load_data(csv_file: str) -> pd.DataFrame:
    return pd.read_csv(f"../trade_demo/datasets/{csv_file}")[0:10]

def get_user_details(unique_email: str) :
    return {
        "name": "Sheldon Cooper",
        "email": unique_email,
        "password": "bazinga",
        "budget": 10,
    }

unique_email = f"{uuid.uuid4()}@caltech.edu"

# Canada
ca_root = sy.login(email="info@openmined.org", password="changethis", port=DOMAIN1_PORT)
ca_data = load_data(csv_file="ca - feb 2021.csv")

data = np.array([[0,0],[0,0]])
# canada_trade = (
#     (np.array(list(ca_data["Trade Value (US$)"])) / 1000)[0:10]
# ).astype(np.int64)
canada_trade = np.array([[0,0],[0,0]])
trade_partners = ((list(ca_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(DataSubject(name=trade_partners[i]))

sampled_canada_dataset = sy.Tensor(canada_trade)
sampled_canada_dataset.public_shape = sampled_canada_dataset.shape
sampled_canada_dataset = sampled_canada_dataset.private(
      0, 3, data_subjects=[entities[0].name]*canada_trade.shape[0]
).tag("trade_flow")

# load dataset
ca_root.load_dataset(
    assets={"Canada Trade": sampled_canada_dataset},
    name="Canada Trade Data - First few rows",
    description=(
        "A collection of reports from Canada's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
    
)

assert len(ca_root.datasets) > 0



# Italy
it_root = sy.login(email="info@openmined.org", password="changethis", port=DOMAIN2_PORT)
it_data = load_data(csv_file="it - feb 2021.csv")

# italy_trade = ((np.array(list(it_data["Trade Value (US$)"])) / 1000)[0:10]).astype(
#     np.int64
# )
italy_trade = np.array([[0,0],[0,0]])+1
trade_partners = ((list(it_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(DataSubject(name="Other Asia, nes"))

# Upload a private dataset to the Domain object, as the root owner
sampled_italy_dataset = sy.Tensor(italy_trade)
sampled_italy_dataset.public_shape = sampled_italy_dataset.shape
sampled_italy_dataset = sampled_italy_dataset.private(
    0, 3, data_subjects=[entities[0].name]*canada_trade.shape[0]
).tag("trade_flow")

it_root.load_dataset(
    assets={"Italy Trade": sampled_italy_dataset},
    name="Italy Trade Data - First few rows",
    description=(
        "A collection of reports from iStat's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
    
)




Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


/tmp/ipykernel_2881670/2162849251.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"../trade_demo/datasets/{csv_file}")[0:10]
kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/azureuser


Loading dataset... checking asset types...                              

**Warning**: Blob Storage is disabled on this domain. Switching to database store.

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into italy... done!
Loading dataset... checking asset types...                              

**Warning**: Blob Storage is disabled on this domain. Switching to database store.

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


/tmp/ipykernel_2881670/2162849251.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"../trade_demo/datasets/{csv_file}")[0:10]


In [ ]:
from syft.core.node.common.node_service.user_manager.user_messages import (
    UpdateUserMessage,
)

# Upgrade admins budget
content = {"user_id": 1, "budget": 9_999_999}
p1._perform_grid_request(grid_msg=UpdateUserMessage, content=content)
p2._perform_grid_request(grid_msg=UpdateUserMessage, content=content)

print(p1.privacy_budget)
print(p2.privacy_budget)

In [ ]:
x = p1.datasets[-1]["Canada Trade"]
y = p2.datasets[-1]["Italy Trade"]

In [ ]:
# working operations
ops = ["add","sub","mul","matmul","lt","gt","eq","ne","ge","le"]
idx = 0 # points to the op
op = getattr(operator,ops[idx])
res = op(x,y)
#For comparison increase the timeout
res.block_with_timeout(10)
val = res.get_copy()

In [ ]:
val == op(sampled_canada_dataset.decode(),sampled_italy_dataset.decode())

In [ ]:
print(val) #FPT result
print(op(sampled_canada_dataset.decode(),sampled_italy_dataset.decode())) #Acutal result

In [ ]:
# DEBUG CELL
# print((sampled_canada_dataset).decode())
# print((sampled_italy_dataset).decode())
# print(op(sampled_canada_dataset.decode(),sampled_italy_dataset.decode()).round())
# print(res.get_copy().round())


In [ ]:
# p = res
# public_result = p.publish(sigma=1)

In [ ]:
# pb_res = public_result.get_copy()

In [ ]:
# print(pb_res)
# print(p.get_copy())

In [ ]:
print(res.get_copy())

In [ ]:
v = exp(res)

In [ ]:
v.get_copy()

In [ ]:
v1 = -res

In [ ]:
v1.get_copy()

In [ ]:
print(v.get_copy())
print(poovathal.get_copy())

In [ ]:
x = res * 10

In [ ]:
t1 = res<res

In [ ]:
t2 = res*t1

In [ ]:
print(t1.get_copy())
print(res.get_copy())
# t1.get_copy()*res.get_copy()

In [ ]:
res.get_copy()

In [ ]:
#Below are debug cells
a=np.array([ -789925275 ,  798267317, -1708906764,  1806674886,  1165022425,   480477472,
    54663083  ,  78462192,  -974424744, -1538872148],dtype=np.int32)
b=np.array([  789925275,  -798266317 , 1708920764, -1806674886, -1165022425,  -480477472,
   -54663083  , -78459192  , 974424744 , 1538872148],dtype=np.int32)

In [ ]:
(a+b)/1000

In [ ]:
sampled_canada_dataset.decode()

In [ ]:
a=np.array([-1362805391  ,1471796679 , 2126726498   ,874113086 , -532526475, -1815214451,
  1602118491,  1561978677 , -970006488 ,  792178136],dtype=np.int32)

b=np.array([ 1362805391, -1470796679 ,-2084726498  ,-874113086  , 532526475  ,1815214451,
 -1602118491 ,-1558978677 ,  970006488  ,-792178136],dtype=np.int32)


In [ ]:
a= np.array([ 1300758168, -1917863768, -1667359440,  2060242384 ,  -49469704 ,-1556715208,
   -95689592 ,   94451448 , -656120896, -1904153536],dtype=np.int32)

b=np.array( [-1300758168 ,-1377103528  , 717686480, -2060242384,    49469704,  1556715208,
    95689592, -1389418744 ,  656120896  ,1904153536],dtype=np.int32)

In [ ]:
res.child[0].get_copy()+res.child[1].get_copy()

In [ ]:
def value(a):
    v1 = a.child[0].get_copy()
    v2 = a.child[1].get_copy()
    v = v1+v2
    return v.child

In [ ]:
v1 = v.child[0].get_copy()
v2 = v.child[1].get_copy()

In [ ]:
t1 = v1.child.child.child.child
t2 = v2.child.child.child.child

In [ ]:
(t1+t2)/ ((2**16)**2)

In [ ]:
print(t1)
print(t2)

In [ ]:
x1 = t1[0][0]
x2 = t2[0][0]

In [ ]:
(x1//2**16) + (x2//2**16)

In [ ]:
x1//2**16

In [ ]:
x2//2**16

In [ ]:
x1//2**16

In [ ]:
x2//2**16

In [ ]:
import torch

In [ ]:
w1 = torch.from_numpy(np.array([446545456455]))
w2 = torch.from_numpy(np.array([446545456455]))

In [ ]:
print(w1)
print(w1/2**16)
print(w1//2**16)
print(torch.div(w1,2**16,rounding_mode="trunc"))

In [ ]:
print(w2)
print(w2/2**16)
print(w2//2**16)

In [ ]:
np.floor_divide(t1,2**16)

In [ ]:
np.trunc(x1/2**16).astype(np.int64)

In [ ]:
val = sampled_canada_dataset.gamma

In [ ]:
val

In [ ]:
t = val.send(p1)

In [ ]:
v = t+t

In [ ]:
v.get_copy()

In [ ]:
mpc = x+y

In [6]:
t1 = sy.Tensor(sampled_canada_dataset.child.gamma)
t2 = sy.Tensor(sampled_italy_dataset.child.gamma)

In [7]:
v1 = t1.send(p1)
v2 = t2.send(p2)


**Warning**: Blob Storage is disabled on this client node. Switching to database store.

**Warning**: Blob Storage is disabled on this client node. Switching to database store.


In [ ]:
res = v1+v2

In [9]:
res.get_copy()

array([[1., 1.],
       [1., 1.]])

In [4]:
val  = sampled_canada_dataset.child.gamma

In [5]:
sy.serialize(val)

b'\x00\x00\x00\x00\xd8\x00\x00\x00\x00\x00\x00\x00\x01\x00\x08\x00\x01\x00\x00\x00\x00\x00\x00\x00\x1d\x00\x00\x00\xba\x00\x00\x00%\x00\x00\x00\xba\x06\x00\x00\x8d\x00\x00\x00\xba\x01\x00\x00\xa4\x00\x00\x00\x00\x00\x02\x00\xf0\x00\x00\x00\x00\x00\x02\x00Y\x01\x00\x00\xe2\x0e\x00\x00E\x02\x00\x00\xf2\x0e\x00\x001\x03\x00\x00R\x00\x00\x00capnp:GammaTensor:capnp\x00\x10\x1eP\x02\x02\x11\x02\x10\x01\x011\x05\x02\x01\x11\x11\x0e\xffcapnp:Fi\x03xedPrecisionTensor:capnp1\x01\xea\x04\xff\x10\x17@\x02\x11\x05\x0eQ\x12D\x01\x011\x01\xfa\x03\xbf(\xb5/\xfd` \xad\xfd\x03\x92\xc5\x0f\x1b\xa0\'\xffi\x0csUUUU_\rUU\xd57>o?\xfe\xd3\x91m\x19\xf9\x9f\x9c\xddR\'\x8c\xad+\xe6h\xc9-\xf7\xe1\xb4r\x1a\x1cq\x05\x1d\xcd"hx\\H\xc8\xd7\xba\x80\x81\x84\xd5|q\xce9\x01\x16\x00\x01\x07\xab\x08\xce\\\x18\xbdV\xb0\x06Hw\xa82\xb9\x85\xa2\xda\x9dqg\xc6b\xb9\x0b*\xe3\xca\xcct+\x1e\xb9\x012/\xe29\xfa\xa8\xab`\xc3]k\x18@\x02\x00\x03 \x01\x11\x012\x1f\x1fint64\x00\n\tprotobuf:\x12\x14syft.lib.python.Dict\x1a\x14\x1a\x12\n\x1